In [ ]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.1 MB/s eta 0:00:00


In [ ]:
!apt update > /dev/null
!apt install chromium-chromedriver -y > /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin




W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
!pip install cloudscraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install cfscrape

In [ ]:
!pip show cfscrape
!pip show urllib3

Name: cfscrape
Version: 2.1.1
Summary: A simple Python module to bypass Cloudflare's anti-bot page. See https://github.com/Anorov/cloudflare-scrape for more information.
Home-page: https://github.com/Anorov/cloudflare-scrape
Author: Anorov
Author-email: anorov.vorona@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.11/dist-packages
Requires: requests
Required-by: 
Name: urllib3
Version: 2.3.0
Summary: HTTP library with thread-safe connection pooling, file post, and more.
Home-page: 
Author: 
Author-email: Andrey Petrov <andrey.petrov@shazow.net>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: distributed, kaggle, requests, selenium, sentry-sdk


In [ ]:
!pip install requests-html

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import string
import cloudscraper

base_url = "https://startupmahakumbh.org/exhibitor_directory/exhi_list_pub.php"
event_params = {
    "event_name": "sm",
    "event_year": "2025",

}

all_orgs = []

fields = [
    "Name of the Organization",
    "Website",
    "Address",
    "Contact Person",
    "Designation",
    "Contact Details",
    "Profile"
]

scraper = cloudscraper.create_scraper()

for letter in list(string.ascii_uppercase) + ["#"]:
    print(f"\n Scraping filter: {letter}")
    page = 1

    while True:
        event_params["filter"] = letter
        event_params["page"] = page

        try:
            response = scraper.get(base_url, params=event_params)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            org_blocks = soup.find_all("p")
            if not org_blocks:
                print(f" No organizations found on page {page} of filter '{letter}'")
                break

            org_data = {}
            orgs_found = 0

            for p_tag in org_blocks:
                strong = p_tag.find("strong")
                if strong:
                    label = strong.text.strip().replace(":", "")

                    if label == "Name of the Organization":
                        details_div = p_tag.find_parent("div", class_="details")
                        if details_div:
                            anchor = details_div.find("a", href=True)
                            if anchor and not "/cdn-cgi/l/email-protection" in anchor["href"]:
                                org_data["Website"] = anchor["href"].strip()
                            else:
                                org_data["Website"] = "Website not available"

                    if label == "Contact Details":
                        value = strong.next_sibling.strip() if strong.next_sibling else "Not available"
                        org_data["Contact Details"] = value
                    else:
                        value = strong.next_sibling.strip() if strong.next_sibling else ""
                        org_data[label] = value

                    if label == "Profile":
                        row = [org_data.get(field, "") for field in fields]
                        all_orgs.append(row)
                        orgs_found += 1
                        org_data = {}

            print(f" Page {page}: {orgs_found} organizations scraped.")
            page += 1

        except requests.exceptions.RequestException as e:
            print(f" Error on filter '{letter}', page {page}: {e}")
            break

csv_filename = "startup_mahakumbh_organizations_full_with_contact_details.csv"
with open(csv_filename, "w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(fields)
    writer.writerows(all_orgs)

print(f"\n Scraping complete! {len(all_orgs)} organizations saved to '{csv_filename}'")

from google.colab import files
files.download(csv_filename)



🔍 Scraping filter: A
✅ Page 1: 15 organizations scraped.
✅ Page 2: 15 organizations scraped.
✅ Page 3: 15 organizations scraped.
✅ Page 4: 15 organizations scraped.
✅ Page 5: 15 organizations scraped.
✅ Page 6: 15 organizations scraped.
✅ Page 7: 15 organizations scraped.
✅ Page 8: 15 organizations scraped.
✅ Page 9: 15 organizations scraped.
✅ Page 10: 15 organizations scraped.
✅ Page 11: 15 organizations scraped.
✅ Page 12: 15 organizations scraped.
✅ Page 13: 15 organizations scraped.
✅ Page 14: 15 organizations scraped.
✅ Page 15: 15 organizations scraped.
✅ Page 16: 11 organizations scraped.
❌ No organizations found on page 17 of filter 'A'

🔍 Scraping filter: B
✅ Page 1: 15 organizations scraped.
✅ Page 2: 15 organizations scraped.
✅ Page 3: 15 organizations scraped.
✅ Page 4: 15 organizations scraped.
✅ Page 5: 15 organizations scraped.
✅ Page 6: 15 organizations scraped.
✅ Page 7: 15 organizations scraped.
✅ Page 8: 13 organizations scraped.
❌ No organizations found on page 9 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(soup.prettify())

<meta charset="utf-8"/>
<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   Exhibitor Directory
  </title>
  <link href="assets/css/styles.min.css" rel="stylesheet"/>
  <link href="assets/css/exhi_styles.css" rel="stylesheet"/>
  <link href="assets/images/logos/favicon.png" rel="shortcut icon" type="image/png">
   <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet"/>
   <style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:ital,wght@0,100;0,300;0,400;0,500;0,700;0,900;1,100;1,300;1,400;1,500;1,700;1,900&display=swap');

                body {
                    font-family: "Roboto", sans-serif;
                    font-weight: 500;
                    font-style: normal;
                    margin: 0;
                    padding: 0;
                    background-color: #ffffff;
                }
                @me